In [711]:
%run ./common.ipynb


filepath='/Users/u1002018/Library/CloudStorage/OneDrive-SharedLibraries-FootLocker/Global Technology Services - DASH Doc Library/SOLE/'
# filepath='./output/'


datestr = ""

g_Df = pd.DataFrame (columns=['key','FixVersion', 
 'Priority',
 'Acceptance Criteria',
 'Severity Level',
 'Labels',
 'EpicLink',
 'Issuelinks',
 'Status',
 'Components',
 'IssueCreator',
 'creator.emailAddress',
 'creator.displayName',
 'SubTasks',
 'Reporter.name',
 'reporter.emailAddress',
 'reporter.displayName',
 'issueType',
 'projectKey',
 'projectName',
 'created_ts',
 'updated_ts',
 'ParentLink',
 'summary',
 'StoryPoint',
 'Sprint', 
 'AssigneeName',
 'AssigneeEmailAddress',
 'AssigneeDisplayName',
 'ResolutionDescription',
 'ResolutionName',
 'parent.Summary',
 'parent.key',
 'parent.status',
 'parent.priority',
 'Defect Type',
 'Test Cycle',
 'Bug Severity',
 'Sub-track',
 'RICE Object Type',
 'Complexity',
 'Scope',
 'Baseline Scope',
 'Sprint_state',
 'Sprint_name',
 'Sprint_startDate',
 'Sprint_endDate',
 'Sprint_completeDate',        
 'Sprint_activatedDate',
 'Sprint_sequence',
 'Sprint_goal',
 'Intial SOW',
 'After Global Design',
 'The Rudy Special'
 ])



In [712]:
global logger
logger = setup_logger()
logger.info("=======================================")
logger.info("Script start accumulating data from JIRA")

2024-11-27 15:40:43,331 - root - INFO - =======================================
2024-11-27 15:40:43,331 - root - INFO - Script start accumulating data from JIRA


In [713]:
def data_clean(dataframe):
    if dataframe is not None and not dataframe.empty:
       dataframe= dataframe.filter(['key','fields.fixVersions',
                    'fields.priority.name',
                    'fields.customfield_31601',
                    'fields.customfield_12402.value',
                    'fields.labels',
                    'fields.customfield_10006',
                    # 'fields.issuelinks',
                    'fields.status.name',
                    'fields.components',
                    'fields.creator.name',
                    'fields.creator.emailAddress',
                    'fields.creator.displayName',
                    'fields.subtasks',
                    'fields.reporter.name',
                    'fields.reporter.emailAddress',
                    'fields.reporter.displayName',
                    'fields.issuetype.name',
                    'fields.project.key',
                    'fields.project.name',
                    'fields.created',
                    'fields.updated',
                    'fields.customfield_12823',
                    'fields.summary',
                    'fields.customfield_10002',
                    'fields.customfield_10004',
                    'fields.assignee.name',
                    'fields.assignee.emailAddress',
                    'fields.assignee.displayName',
                    'fields.resolution.description',
                    'fields.resolution.name',
                    'fields.parent.fields.summary',
                    'fields.parent.key',
                    'fields.parent.fields.status.name',
                    'fields.parent.fields.priority.name',
                    'fields.customfield_32101.value',
                    'fields.customfield_32302.value',
                    'fields.customfield_11605.value',
                    'fields.customfield_32901',
                    'fields.customfield_16902' ,
                    'fields.customfield_32900.value' ,
                    'fields.customfield_27503' ,
                    'fields.customfield_16902.value',
                    'fields.customfield_33000']).rename(columns={'fields.fixVersions': 'FixVersion', 
                                  'fields.priority.name':'Priority',
                                  'fields.customfield_31601': 'Acceptance Criteria',
                                  'fields.customfield_12402.value':'Severity Level',
                                  'fields.labels':'Labels',
                                  'fields.customfield_10006':'EpicLink',
                                #   'fields.issuelinks': 'Issuelinks',
                                  'fields.status.name': 'Status',
                                  'fields.components':'Components',
                                  'fields.creator.name':'IssueCreator',
                                  'fields.creator.emailAddress':'creator.emailAddress',
                                  'fields.creator.displayName': 'creator.displayName',
                                  'fields.subtasks':'SubTasks',
                                  'fields.reporter.name':'Reporter.name',
                                  'fields.reporter.emailAddress':'reporter.emailAddress',
                                  'fields.reporter.displayName': 'reporter.displayName',
                                  'fields.issuetype.name':'issueType',
                                  'fields.project.key':'projectKey',
                                  'fields.project.name':'projectName',
                                  'fields.created':'created_ts',
                                  'fields.updated':'updated_ts',
                                  'fields.customfield_12823': 'ParentLink',
                                  'fields.summary':'summary',
                                  'fields.customfield_10002':'StoryPoint',
                                  'fields.customfield_10004':'Sprint',                                  
                                  'fields.assignee.name':'AssigneeName',
                                  'fields.assignee.emailAddress':'AssigneeEmailAddress',
                                  'fields.assignee.displayName':'AssigneeDisplayName',
                                  'fields.resolution.description':'ResolutionDescription',
                                  'fields.resolution.name':'ResolutionName',
                                  'fields.parent.fields.summary': 'parent.Summary',
                                  'fields.parent.key': 'parent.key',
                                  'fields.parent.fields.status.name': 'parent.status',
                                  'fields.parent.fields.priority.name':'parent.priority',
                                  'fields.customfield_32101.value':'Defect Type',
                                  'fields.customfield_32302.value':'Test Cycle',
                                  'fields.customfield_11605.value':'Bug Severity',
                                  'fields.customfield_32901':'Sub-track',
                                  'fields.customfield_32900.value' : 'RICE Object Type',
                                  'fields.customfield_27503' : 'Complexity',
                                  'fields.customfield_16902.value':'Scope',
                                  'fields.customfield_33000':'Baseline Scope'
                                  })
    return dataframe


In [714]:
def fetch(issueType, project):
    global fetch_df 
    logger.info(f"Fetch {issueType}...")
    attribute =' &expand=projects.issuetypes.fields'
    searchQry=f'?jql= project in ("{project}") and issueType="{issueType}" '

    # if cond_flg == True: 
    #     searchQry=f"{searchQry} and status not in (Cancelled, Done)  {attribute}"
    # else:
    searchQry=f"{searchQry} {attribute}"

    fetch_df = getDataSet(searchQry)
    
    # if searchQry_DS:
    #     fetch_df=pd.json_normalize(searchQry_DS)
    # else:
    #     logger.error(f'No records found under {project} - {issueType} category!')
    
    return data_clean(fetch_df)    
    

In [715]:
pd.set_option('display.max_columns', None)

In [716]:
issueLists = [
              {'issueType':"Portfolio Initiative",'cond_flg':False},
              {'issueType':"Product Initiative", 'cond_flg':False},
              {'issueType':"Epic", 'cond_flg':False},
              {'issueType':"Story", 'cond_flg':False},
              {'issueType':"Task", 'cond_flg':False},
              {'issueType':"Sub-task", 'cond_flg':False},
              {'issueType':"Bug", 'cond_flg':False},
              {'issueType':"Incident", 'cond_flg':False},
              {'issueType':"Production Defects", 'cond_flg':False},
              {'issueType':"Defect", 'cond_flg':False},
              {'issueType':"Issue", 'cond_flg':False},
              {'issueType':"Test", 'cond_flg':False}
             ]
projects =["SOLEFIN", "SOLMerch"]
# projects =["SOLEFIN"]
for project in projects:
    for issueList in issueLists:
        df=fetch(issueList['issueType'], project)
        if df is not None and not df.empty:
            g_Df = pd.concat([g_Df, df ], ignore_index=True)
    

2024-11-27 15:40:43,480 - root - INFO - Fetch Portfolio Initiative...
2024-11-27 15:40:43,481 - root - INFO - Getting report results...
2024-11-27 15:40:43,481 - root - INFO - https://jira.footlocker.com/rest/api/2/search?jql= project in ("SOLEFIN") and issueType="Portfolio Initiative"   &expand=projects.issuetypes.fields&maxResults=2000&startAt=0
2024-11-27 15:40:44,001 - root - INFO - <Response [200]>
2024-11-27 15:40:44,002 - root - INFO - Report results received...
2024-11-27 15:40:44,003 - root - INFO - HTTP 200 - OK
2024-11-27 15:40:44,003 - root - INFO - total: 0
2024-11-27 15:40:44,004 - root - ERROR - No records found....
2024-11-27 15:40:44,081 - root - INFO - Fetch Product Initiative...
2024-11-27 15:40:44,082 - root - INFO - Getting report results...
2024-11-27 15:40:44,083 - root - INFO - https://jira.footlocker.com/rest/api/2/search?jql= project in ("SOLEFIN") and issueType="Product Initiative"   &expand=projects.issuetypes.fields&maxResults=2000&startAt=0
2024-11-27 15:4

In [717]:
g_Df

,key,FixVersion,Priority,Acceptance Criteria,Severity Level,Labels,EpicLink,Issuelinks,Status,Components,IssueCreator,creator.emailAddress,creator.displayName,SubTasks,Reporter.name,reporter.emailAddress,reporter.displayName,issueType,projectKey,projectName,created_ts,updated_ts,ParentLink,summary,StoryPoint,Sprint,AssigneeName,AssigneeEmailAddress,AssigneeDisplayName,ResolutionDescription,ResolutionName,parent.Summary,parent.key,parent.status,parent.priority,Defect Type,Test Cycle,Bug Severity,Sub-track,RICE Object Type,Complexity,Scope,Baseline Scope,Sprint_state,Sprint_name,Sprint_startDate,Sprint_endDate,Sprint_completeDate,Sprint_activatedDate,Sprint_sequence,Sprint_goal,Intial SOW,After Global Design,The Rudy Special,fields.customfield_16902
0,SOLEFIN-12485,[],Medium,None,Unassigned,[],None,NaN,Done,[],v008647,v008647@footlocker.com,Masi Najafizehtab - v008647,[],v008647,v008647@footlocker.com,Masi Najafizehtab - v008647,Epic,SOLEFIN,SOLE Finance,2024-10-30T06:40:22.000-0500,2024-11-13T10:52:17.000-0600,None,R1.E2E.12.Master Data Push to PCMCS Application,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,SOLEFIN-12484,[],Medium,None,Unassigned,[],None,NaN,Done,[],v008647,v008647@footlocker.com,Masi Najafizehtab - v008647,[],v008647,v008647@footlocker.com,Masi Najafizehtab - v008647,Epic,SOLEFIN,SOLE Finance,2024-10-30T06:40:21.000-0500,2024-11-13T10:23:08.000-0600,None,R1.E2E.11.Master Data Push to EPBCS Application,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,SOLEFIN-12483,[],Medium,None,Unassigned,[],None,NaN,Done,[],v008647,v008647@footlocker.com,Masi Najafizehtab - v008647,[],v008647,v008647@footlocker.com,Masi Najafizehtab - v008647,Epic,SOLEFIN,SOLE Finance,2024-10-30T06:40:21.000-0500,2024-11-13T10:31:49.000-0600,None,R1.E2E.10.Master Data Push to FCCS Application,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,SOLEFIN-12482,[],Medium,None,Unassigned,[],None,NaN,Done,[],v008647,v008647@footlocker.com,Masi Najafizehtab - v008647,[],v008647,v008647@footlocker.com,Masi Najafizehtab - v008647,Epic,SOLEFIN,SOLE Finance,2024-10-30T06:40:20.000-0500,2024-11-13T11:07:42.000-0600,None,R1.E2E.09.Master Data Push to GL Application,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,SOLEFIN-12481,[],Medium,None,Unassigned,[],None,NaN,Done,[],v008647,v008647@footlocker.com,Masi Najafizehtab - v008647,[],v008647,v008647@footlocker.com,Masi Najafizehtab - v008647,Epic,SOLEFIN,SOLE Finance,2024-10-30T06:40:19.000-0500,2024-11-13T10:26:35.000-0600,None,R1.E2E.08.Metadata Application Backup,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12540,SOLMERCH-3464,[],Medium,None,Unassigned,"[INT_273, ITEM, PIM, SIT1]",SOLMERCH-3021,NaN,Done,[],v009344,v009344@footlocker.com,Meenakshi Priya Arumugam - v009344,[],v009344,v009344@footlocker.com,Meenakshi Priya Arumugam - v009344,Test,SOLMERCH,SOLE Merch,2024-10-16T07:11:49.000-0500,2024-11-20T07:29:25.000-0600,None,Create an Item Reclassification in PIM for an ...,NaN,None,v009344,v009344@footlocker.com,Meenakshi Priya Arumugam - v009344,Work has been completed on this issue.,Done,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12541,SOLMERCH-3463,[],Medium,None,Unassigned,"[INT_273, ITEM, PIM, SIT1]",SOLMERCH-3021,NaN,Done,[],v009344,v009344@footlocker.com,Meenakshi Priya Arumugam - v009344,[],v009344,v009

In [718]:
# g_Df = pd.concat([g_Df, df ], ignore_index=True)

In [719]:
# logger.info("Fetch SOLFIN sub-tasks...")
# searchQry=f'?jql= project in ("SOLEFIN") and issueType="Sub-task" and  createdDate <= startOfYear("6M")  &maxResults=5000&&expand=projects.issuetypes.fields'
# searchQry_DS = getDataSet(searchQry)
# if searchQry_DS:
#     SOLEFIN_lessthanSix_subTask_Df=pd.json_normalize(searchQry_DS)
# else:
#     logger.error("SOLEFIN_lessthanSix_subTask_Df was added to the data set")

In [720]:
# searchQry=f'?jql= project in ("SOLEFIN") and createdDate > startOfYear("6M") and issueType="Sub-task" and status not in (done) &maxResults=5000&&expand=projects.issuetypes.fields'
# searchQry_DS = getDataSet(searchQry)
# if searchQry_DS:
#     SOLEFIN_SixPlus_subTask_Df=pd.json_normalize(searchQry_DS)
# else:
#     logger.error("SOLEFIN_SixPlus_subTask_Df was added to the data set")

In [721]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 500)


In [722]:
g_Df['Sprint'] = g_Df['Sprint'].apply(lambda x: convert_to_key_value(x))

g_Df['Sprint_state'] = g_Df['Sprint'].apply(lambda x: sprintvalue(x,'state') if x is not None else "NA" )
g_Df['Sprint_name']= g_Df['Sprint'].apply(lambda x: sprintvalue(x,'name') if x is not None else "NA" )
g_Df['Sprint_startDate']= g_Df['Sprint'].apply(lambda x: sprintvalue(x,'startDate') if x is not None else "NA" )
g_Df['Sprint_endDate']= g_Df['Sprint'].apply(lambda x: sprintvalue(x,'endDate') if x is not None else "NA" )
g_Df['Sprint_completeDate']= g_Df['Sprint'].apply(lambda x: sprintvalue(x,'completeDate') if x is not None else "NA" )
g_Df['Sprint_activatedDate']= g_Df['Sprint'].apply(lambda x: sprintvalue(x,'activatedDate') if x is not None else "NA" )
# sole_tracker['Sprint_sequence']= epic_Df['Sprint'].apply(lambda x: sprintvalue(x,'sequence') if x is not None else "NA" )
g_Df['Sprint_goal']= g_Df['Sprint'].apply(lambda x: sprintvalue(x,'goal') if x is not None else "NA" )
g_Df= g_Df.drop(columns=['Sprint'])
# epic_Df['FixVersions'][4207][0]['name']
g_Df['FixVersion']= g_Df['FixVersion'].apply(lambda x: keyvalue(x,'name') if x is not None else "" )

In [723]:
def getBaselineLst(element):
   baselinelst =[]
   if element != None:
      print(element)
      if  element != 'NA':
         for idx in element:
            baselinelst.append(idx['value'])
   return baselinelst

In [724]:
g_Df=g_Df.fillna("NA")
g_Df['Baseline Scope']=g_Df['Baseline Scope'].apply(lambda x: getBaselineLst(x) )
g_Df['Intial SOW']= g_Df['Baseline Scope'].apply(lambda x: {True:'YES',False:'NO'}[x.__contains__('Baseline 1')] )
g_Df['After Global Design']= g_Df['Baseline Scope'].apply(lambda x: {True:'YES',False:'NO'}[x.__contains__('Baseline 2')] )
g_Df['The Rudy Special']= g_Df['Baseline Scope'].apply(lambda x: {True:'YES',False:'NO'}[x.__contains__('Baseline 3')] )

NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
N

In [725]:
g_Df[g_Df['Intial SOW']=='YES'][['Intial SOW','After Global Design','The Rudy Special','Baseline Scope']]

,Intial SOW,After Global Design,The Rudy Special,Baseline Scope
8912,YES,YES,YES,"[Baseline 1, Baseline 2, Baseline 3]"
8913,YES,YES,YES,"[Baseline 1, Baseline 2, Baseline 3]"
8914,YES,YES,YES,"[Baseline 1, Baseline 2, Baseline 3]"
8915,YES,YES,YES,"[Baseline 1, Baseline 2, Baseline 3]"
8916,YES,YES,YES,"[Baseline 1, Baseline 2, Baseline 3]"
8917,YES,YES,YES,"[Baseline 1, Baseline 2, Baseline 3]"
8918,YES,YES,YES,"[Baseline 1, Baseline 2, Baseline 3]"
8919,YES,YES,YES,"[Baseline 1, Baseline 2, Baseline 3]"
8920,YES,YES,YES,"[Baseline 1, Baseline 2, Baseline 3]"
8921,YES,YES,YES,"[Baseline 1, Baseline 2, Baseline 3]"


In [726]:
logger.info("Upload file to sharepoint...")

try:
    g_Df[g_Df['issueType'] == 'Portfolio Initiative'].to_csv(os.path.join(filepath, 'Portfolio_Initiative.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Product Initiative'].to_csv(os.path.join(filepath, 'Product_Initiative.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Epic'].to_csv(os.path.join(filepath, 'Epic.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Story'].to_csv(os.path.join(filepath, 'Story.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Task'].to_csv(os.path.join(filepath, 'Task.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Sub-task'].to_csv(os.path.join(filepath, 'Subtasks.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Test'].to_csv(os.path.join(filepath, 'Test.csv'), index=False)
    g_Df.loc[g_Df['issueType'].isin(['Bug','Epic','Incident','Production Defects','Defect','Issue'])].to_csv(os.path.join(filepath, 'Bug.csv'), index=False)
except Exception as e:  
    logger.error(f"Failed to upload files to {filepath}: {e}")
    pass


logger.info("Files uploaded")
logger.info("Script ended accumulating data from JIRA")
logger.info("=======================================")

2024-11-27 15:46:14,258 - root - INFO - Upload file to sharepoint...
2024-11-27 15:46:14,773 - root - INFO - Files uploaded
2024-11-27 15:46:14,773 - root - INFO - Script ended accumulating data from JIRA
2024-11-27 15:46:14,774 - root - INFO - =======================================


In [727]:
# g_Df[g_Df['projectKey'] != 'SOLMERCH']

In [728]:
# sprintvalue(epic_Df['fields.customfield_10004'][0],'completeDate')

In [729]:
# projid=("MRC-","PTP-","ITC-", "RTR-", "PTC-", "Tax-", "ARC-", "CCR-", "MDM-", "OFS-", "LRP-", ",RPT-")

# sole_tracker = pd.DataFrame(columns=["ProjectId","JiraID","IssueType","RiceId","RiceName",
#  "storyPoints","status"
#  ,'Sprint_state',
#  'Sprint_name',
#  'Sprint_startDate',
#  'Sprint_endDate',
# 'Sprint_completeDate',        
#  'Sprint_activatedDate',
#  'Sprint_sequence',
#  'Sprint_goal',
#  'labels',
#  'components'
#  ])

# sole_tracker["ProjectId"]=epic_Df['fields.project.key']
# sole_tracker["JiraID"] = epic_Df['key'],

# sole_tracker["IssueType"] = epic_Df['fields.issuetype.name']
# sole_tracker["RiceId"] = epic_Df[epic_Df['fields.summary'].str.startswith(projid)]['fields.summary'].str.partition(' ')[0]
# sole_tracker["RiceName"] = epic_Df['fields.summary'].apply(lambda row: row.partition(' ')[2] if row.startswith(projid) else row)
# sole_tracker["status"] =epic_Df['fields.status.name']
# sole_tracker["storyPoints"] =epic_Df['fields.customfield_10002']
# sole_tracker["labels"] =epic_Df['fields.labels']
# sole_tracker["components"] =epic_Df['fields.components']
# # sole_tracker['Sprint_state']=epic_Df['fields.customfield_10004'][0]['state']
# sole_tracker['Sprint_state'] = epic_Df['fields.customfield_10004'].apply(lambda x: sprintvalue(x,'state') if x is not None else "NA" )
# sole_tracker['Sprint_name']= epic_Df['fields.customfield_10004'].apply(lambda x: sprintvalue(x,'name') if x is not None else "NA" )
# sole_tracker['Sprint_startDate']= epic_Df['fields.customfield_10004'].apply(lambda x: sprintvalue(x,'startDate') if x is not None else "NA" )
# sole_tracker['Sprint_endDate']= epic_Df['fields.customfield_10004'].apply(lambda x: sprintvalue(x,'endDate') if x is not None else "NA" )
# sole_tracker['Sprint_completeDate']= epic_Df['fields.customfield_10004'].apply(lambda x: sprintvalue(x,'completeDate') if x is not None else "NA" )
# sole_tracker['Sprint_activatedDate']= epic_Df['fields.customfield_10004'].apply(lambda x: sprintvalue(x,'activatedDate') if x is not None else "NA" )
# sole_tracker['Sprint_sequence']= epic_Df['fields.customfield_10004'].apply(lambda x: sprintvalue(x,'sequence') if x is not None else "NA" )
# sole_tracker['Sprint_goal']= epic_Df['fields.customfield_10004'].apply(lambda x: sprintvalue(x,'goal') if x is not None else "NA" )


# # sole_tracker['IssueType' != 'Epic']

# sole_tracker


In [730]:

# sole_tracker_S["RiceId"] = epic_Df[epic_Df['fields.summary'].str.startswith(('MRC-',"PTP-","ITC-", "RTR-", "PTC-", "Tax-", "ARC-", "CCR-", "MDM-", "OFS-", "LRP-", ",RPT-"))]['fields.summary'].str.partition(' ')[0]
# sole_tracker_S["RiceName"] = epic_Df['fields.summary'].apply(lambda row: row.partition(' ')[2] if row.startswith(('MRC-',"PTP-","ITC-", "RTR-", "PTC-", "Tax-", "ARC-", "CCR-", "MDM-", "OFS-", "LRP-", ",RPT-")) else row)

# sole_tracker[sole_tracker['storyPoints'] >= 1]


In [731]:
# sole_tracker[sole_tracker['IssueType']== 'Story'].to_csv(f'{filepath}Story{datestr}.csv')
# sole_tracker[sole_tracker['IssueType']== 'Epic'].to_csv(f'{filepath}Epic{datestr}.csv')


In [732]:

# 15/* * * * *  /usr/bin/python3 /Users/u1002018/src/PAE/JIRA/py/sole_proj.py

In [733]:

# def getLinkedIssues(key):
#     check= epic_Df[epic_Df['key'] == key].Issuelinks.astype(bool)
#     if check.to_list()[0] == True:
#         issueLink_str=epic_Df[epic_Df['key'] == key]['Issuelinks']
#         issueLink_str=issueLink_str.to_list()[0][0]

#         searchQry=f'?jql= issue in linkedIssues("{key}")'
#         searchQry_DS = getDataSet(searchQry)
#         linkedIssue=pd.json_normalize(searchQry_DS)

#         specficfields = linkedIssue[['key','fields.status.name','fields.priority.name','fields.summary']]
#         specficfields=specficfields.rename(columns={'fields.status.name': 'status','fields.priority.name':'priority', 'fields.summary':'summary'})
#         json_specficfields =specficfields.to_json(orient='records', lines=True)
#         # print(json_specficfields)
#         return json_specficfields
#     else:
#         return ""


In [734]:
# import shutil as sl

# sl.copy(source,destination)